In [17]:
from bs4 import BeautifulSoup
import requests 
from csv import writer
import pandas as pd

In [2]:
#NewEgg is the target for this scrapper
url = 'https://www.newegg.com/Desktop-Graphics-Cards/SubCategory/ID-48?Tid=7709&PageSize=96'
#A header containing you user agent is needed to avoid the bot detection
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}

page = requests.get(url, headers=headers)

#Using soup to scrap the html elements of the web page
soup1 = BeautifulSoup(page.content, 'html.parser')
#Prettify the soup for an easier time manual cheking the data
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

#Finding all the html elements that contain 
#lists = soup2.find_all('div', class_='item-container')

for list in soup2.find_all('div', class_='item-container'):
    title = list.find('a', class_='item-title').get_text().replace('\n', '').replace('RADEON', 'Radeon').strip()#.split()
    price = list.find('li', class_='price-current').get_text().replace('\n', '').strip().split()
    shipping = list.find('li', class_='price-ship').get_text().replace('\n', '').strip()

    #checking if the for loop works and if we get the desired information
    
    #print(title)
    #print(price)
    #print(shipping)
    #info=[title, price, shipping]
    #print(info)

GIGABYTE GeForce RTX 3090 GAMING OC 24G Video Card, GV-N3090GAMING OC-24GD
GIGABYTE Radeon RX 6900 XT GAMING OC Video Card, GV-R69XTGAMING OC-16GD
ASUS ROG Strix GeForce RTX 3050 8GB GDDR6 PCI Express 4.0 Video Card ROG-STRIX-RTX3050-O8G-GAMING
GIGABYTE GAMING OC Radeon RX 6500 XT 4GB GDDR6 PCI Express 4.0 ATX Video Card GV-R65XTGAMING OC-4GD
ASRock OC Formula Radeon RX 6900 XT 16GB GDDR6 PCI Express 4.0 ATX Video Card RX6900XT OCF 16G
GIGABYTE Radeon RX 6700 XT GAMING OC 12G Graphics Card, WINDFORCE 3X Cooling System, 12GB 192-bit GDDR6, GV-R67XTGAMING OC-12GD Video Card
MSI Gaming Radeon RX 6700 XT 12GB GDDR6 PCI Express 4.0 Video Card RX 6700 XT GAMING X 12G
XFX SPEEDSTER SWFT210 Radeon RX 6600 XT 8GB GDDR6 PCI Express 4.0 Core Gaming Graphics Card, AMD RDNA 2 RX-66XT8DFDQ
MSI Ventus GeForce RTX 3050 8GB GDDR6 PCI Express 4.0 Video Card RTX 3050 Ventus 2X 8G OC
MSI Ventus GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 Video Card RTX 3060 Ti VENTUS 2X 8G OCV1 LHR
GIGABYTE Gaming OC PR

In [14]:
def check_price(no_pages):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
    page = requests.get(url, headers=headers)
    
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    gfxcards = []
    
    import datetime
    
    for list in soup2.find_all('div', class_='item-container'):
        title = list.find('a', class_='item-title').get_text().replace('\n', '').replace('RADEON', 'Radeon').strip()
        price = list.find('li', class_='price-current').get_text().replace('\n', '').strip().split()
        shipping = list.find('li', class_='price-ship').get_text().replace('\n', '').strip()
        
        allCards = []
        
        if "GeForce" in title:
            allCards.append("Nvidia")
        elif "Radeon" in title:
            allCards.append("Radeon")
        else:
            allCards.append("Unknown")
        
        allCards.append(title.split()[0])
        
        if "GeForce" in title and 'Ti' not in title:
            allCards.append(title.split("GeForce",1)[1][1:9])
        elif "GeForce" and 'Ti' in title:
            allCards.append(title.split("GeForce",1)[1][1:12])
        elif "Radeon" and 'XT' in title:
            allCards.append(title.split("Radeon",1)[-1][1:11])
        elif 'Radeon' in title and 'XT' not in title:
            allCards.append(title.split("Radeon",1)[1][1:8])
        else:
            allCards.append('Unknown')
        
        allCards.append(price[1])
        
        allCards.append(shipping)
        
        gfxcards.append(allCards)

    return gfxcards

In [15]:
#user input the number of pages to be scrapped, it's recomended to follow the url to check the total number of pages of the NewEggs gpu catalog
no_pages = 15

results = []
for i in range(1, no_pages+1): 
    results.append(check_price(i))
        
#checking the result of the previusly defined function
#print(results)

In [ ]:
#no_pages = input('Number of pages to be scrapped')

#check the price of the GPUs every 24 hours
#while(True):
    #for i in range(1, no_pages+1):
        #results.append(check_price(i))
    #time.sleep(86400)

In [18]:
#unpacking/flattening the lists of list(s of lists)
unpack = lambda l: [item for sublist in l for item in sublist]

#creating a pandas data frame with the data
df = pd.DataFrame(unpack(results),columns=['Manufacturer','Brand','Model','Price', 'Shipping'])
df.to_csv('Graphics card price.csv', index=False, encoding='utf-8')

In [19]:
df = pd.read_csv("Graphics card price.csv")

In [20]:
df.shape

(1440, 5)

In [21]:
df.head()

,Manufacturer,Brand,Model,Price,Shipping
0,Nvidia,GIGABYTE,RTX 3090,"1,999",$9.99 Shipping
1,Radeon,GIGABYTE,RX 6900 XT,"1,099",Free Shipping
2,Nvidia,ASUS,RTX 3050,429,Free Shipping
3,Radeon,GIGABYTE,RX 6500 XT,239,Free Shipping
4,Radeon,ASRock,RX 6900 XT,"1,149",Free Shipping
